In [21]:
# %env NUMBA_DISABLE_JIT=1
%load_ext autoreload

In [27]:
import numpy as np
from libertem_live.detectors.k2is import decode
from libertem_live.utils.net import mcast_socket
from libertem.common.buffers import bytes_aligned

In [3]:
def read_loop_bulk(s, num_packets=64):
    """
    Read `num_packets` at once
    """
    # NOTE: non-IS data is truncated - we only read the first 0x5758 bytes of the message
    packets_per_batch = num_packets
    packet_size = 0x5758
    buf = bytes_aligned(packet_size * packets_per_batch)
    buf_arr = np.frombuffer(buf)
    s.settimeout(1)
    idx = 0
    while True:
        buf_part = buf[idx*packet_size:(idx + 1) * packet_size]
        p = s.recvmsg_into([buf_part])
        assert p[0] == packet_size
        idx += 1
        if idx == packets_per_batch:
            yield (buf, p[1])
            idx = 0
            buf_arr[:] = 0

In [4]:
with mcast_socket(2001, '225.1.1.1', '192.168.10.98', 'enp193s0f0') as s:
    read_iter = read_loop_bulk(s, num_packets=128)
    bulk0 = next(read_iter)

In [5]:
len(bulk0[0])

2862080

In [6]:
len(bulk0[0]) == 0x5758 * 128

True

In [7]:
out = np.zeros((128, 930*16), dtype=np.float32)

In [8]:
%pdb on

Automatic pdb calling has been turned ON


In [34]:
%autoreload 2
inp_mem = bulk0[0]
inp_arr = np.frombuffer(inp_mem, dtype="u1")
decode.decode_bulk_uint12_le(inp=inp_mem, out=out, num_packets=128)

array([[9699506,       0,     240],
       [9699506,       0,     224],
       [9699506,       0,     208],
       [9699506,       0,     192],
       [9699506,       0,     176],
       [9699506,       0,     160],
       [9699506,       0,     144],
       [9699506,       0,     128],
       [9699506,       0,     112],
       [9699506,       0,      96],
       [9699506,       0,      80],
       [9699506,       0,      64],
       [9699506,       0,      48],
       [9699506,       0,      32],
       [9699506,       0,      16],
       [9699506,       0,       0],
       [9699506,     930,     240],
       [9699506,     930,     224],
       [9699506,     930,     208],
       [9699506,     930,     192],
       [9699506,     930,     176],
       [9699506,     930,     160],
       [9699506,     930,     144],
       [9699506,     930,     128],
       [9699506,     930,     112],
       [9699506,     930,      96],
       [9699506,     930,      80],
       [9699506,     930,   

In [45]:
%autoreload 2
inp_mem = bulk0[0]
inp_arr = np.frombuffer(inp_mem, dtype="u1")
%timeit decode.decode_bulk_uint12_le(inp=inp_mem, out=out, num_packets=128)

2.21 ms ± 24.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
4/400

0.01

In [ ]:
%autoreload 2
inp_mem = bulk0[0]
inp_arr = np.frombuffer(inp_mem, dtype="u1")
%timeit decode.decode_bulk_uint12_le(inp=inp_mem, out=out, num_packets=128)

In [11]:
out_2d = out.reshape((4, 1860, 256))

In [12]:
print(out_2d[0, :930, :16].strides)
print(np.zeros((930, 16), dtype=np.float32).strides)

(1024, 4)
(64, 4)


In [13]:
# %%timeit
np.sum(out)

1294298100.0

In [14]:
2/400

0.005

In [15]:
out[:] = 0
out_1 = np.zeros((64, 930*16), dtype=np.float32)
out_2 = np.zeros((64, 930*16), dtype=np.float32)

In [16]:
decode.decode_bulk_uint12_le(inp=bulk0[0], out=out, num_packets=128)
decode.decode_bulk_uint12_le(inp=bulk0[0], out=out_1, num_packets=64)
decode.decode_bulk_uint12_le(inp=bulk0[0][64*0x5758:], out=out_2, num_packets=64)

array([[9699508,       0,     240],
       [9699508,       0,     224],
       [9699508,       0,     208],
       [9699508,       0,     192],
       [9699508,       0,     176],
       [9699508,       0,     160],
       [9699508,       0,     144],
       [9699508,       0,     128],
       [9699508,       0,     112],
       [9699508,       0,      96],
       [9699508,       0,      80],
       [9699508,       0,      64],
       [9699508,       0,      48],
       [9699508,       0,      32],
       [9699508,       0,      16],
       [9699508,       0,       0],
       [9699508,     930,     240],
       [9699508,     930,     224],
       [9699508,     930,     208],
       [9699508,     930,     192],
       [9699508,     930,     176],
       [9699508,     930,     160],
       [9699508,     930,     144],
       [9699508,     930,     128],
       [9699508,     930,     112],
       [9699508,     930,      96],
       [9699508,     930,      80],
       [9699508,     930,   

In [17]:
np.allclose(out[:64], out_1)
np.allclose(out[64:], out_2)

True

In [18]:
%matplotlib widget

In [19]:
import matplotlib.pyplot as plt

In [43]:
plt.figure()
plt.imshow(out_2d[0])
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …